# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment and Imports

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import torch
import time
import os
import numpy as np

from pathlib import Path
from torch.autograd import Variable
from tensorboardX import SummaryWriter

from model import DDPGAgent

In [2]:
from unityagents import UnityEnvironment

In [3]:
env = UnityEnvironment(file_name='./Tennis_Windows_x86_64/Tennis.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [5]:
#edit current run
curr_run = 'run6'
model_dir = Path('./models') 
run_dir = model_dir / curr_run
log_dir = run_dir / 'logs'

#os.makedirs(run_dir)
#os.makedirs(log_dir)


### 3. Set the parameters

in this section we set the parameter to use in our model

In [7]:

# number of training episodes.
number_of_episodes = 4000
episode_length = 50
batch_size = 120

# amplitude of OU noise
# this slowly decreases to 0
noise = 1
noise_reduction = 0.9999

hidden_dim_actor = 256
hidden_dim_critic = 256
gamma=0.95
tau=0.001
lr_actor=0.001
lr_critic=0.001

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


agent_init_params = {'num_in_pol': state_size,
                     'num_out_pol': action_size,
                     'num_in_critic': state_size,
                     'hidden_dim_actor': hidden_dim_actor,
                     'hidden_dim_critic': hidden_dim_critic,
                     'tau':tau,
                     'gamma': gamma,
                    'lr_actor':lr_actor,
                    'lr_critic':lr_critic,
                    'batch_size':batch_size,
                    'max_episode_len':episode_length}



### 4. Train the model

in this section we train the maddpg

In [8]:

agent1 = DDPGAgent(**agent_init_params)
agent2 = DDPGAgent(**agent_init_params)


# load the pretained model

agent_0_weights = run_dir  /  'checkpoint_actor1.pth'
agent_0_critic_weights =  run_dir  /  'checkpoint_critic1.pth'
agent_0_actor_target =  run_dir  /  'checkpoint_actor_target1.pth'
agent_0_critic_target =   run_dir  / 'checkpoint_critic_target1.pth'


agent_1_weights =  run_dir  /  'checkpoint_actor2.pth'
agent_1_critic_weights = run_dir  /  'checkpoint_critic2.pth'
agent_1_actor_target = run_dir  /   'checkpoint_actor_target2.pth'
agent_1_critic_target = run_dir  /   'checkpoint_critic_target2.pth'

agent1.policy.load_state_dict(torch.load(agent_0_weights))
agent1.critic.load_state_dict(torch.load(agent_0_critic_weights))
agent1.target_policy.load_state_dict(torch.load(agent_0_actor_target))
agent1.target_critic.load_state_dict(torch.load(agent_0_critic_target))

agent2.policy.load_state_dict(torch.load(agent_1_weights))
agent2.critic.load_state_dict(torch.load(agent_1_critic_weights))
agent2.target_policy.load_state_dict(torch.load(agent_1_actor_target))
agent2.target_critic.load_state_dict(torch.load(agent_1_critic_target))

In [15]:

agent1 = DDPGAgent(**agent_init_params)
agent2 = DDPGAgent(**agent_init_params)

scores = []
mean_scores = []

agent1.policy.cuda()
agent1.target_policy.cuda()

t_step = 0

for i_episode in range(0, number_of_episodes):
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.vector_observations
    
    agent1.reset_noise()
    agent2.reset_noise()
    score = 0
    
    while True:
        state1 =  state[0]
        state2 = state[1] 
        
        action1 = agent1.act(state1.reshape((1,state1.shape[0])),explore=True)
        action2 = agent2.act(state2.reshape((1,state2.shape[0])),explore=True)
             
        env_info = env.step([action1, action2])[brain_name]
        next_state = env_info.vector_observations
        reward = env_info.rewards
        done = env_info.local_done
        
        next_state1 = next_state[0] 
        next_state2 = next_state[1] 
        
        agent1.step(state1, action1, reward[0],next_state1, done,t_step)
        agent2.step(state2, action2, reward[1],next_state2, done,t_step)
        
        state = next_state
        score += np.max(reward)
        t_step+=1
        
        if np.any(done):
            break
            
    scores.append(score)
    mean = np.mean(scores[-100:])

    print('\rEpisode {}\t Mean Score of both Agents: {:.4f}, replay memory0 {:.3f}, repleay mem1 {:.3f} '.format(i_episode,  mean,len(agent1.replay_buffer),
                                                                                                                                                 len(agent2.replay_buffer)),end="")
    #print('\rstep {}'.format(t_step),end="")
    if i_episode % 200 ==0:
        #logger.add_scalar('agent%i/mean_episode_rewards' % i_episode,  mean_score)
        print('\rEpisode {}\t Mean Score of both Agents: {:.4f}, replay memory0 {:.3f}, repleay mem1 {:.3f} '.format(i_episode,  mean,len(agent1.replay_buffer),
                                                                                                                                                 len(agent2.replay_buffer)))
    
    if i_episode % 100 ==0:
        print('\rEpisode {}\t Mean Score of both Agents: {:.4f}, replay memory0 {:.3f}, repleay mem1 {:.3f} '.format(i_episode,  mean,len(agent1.replay_buffer),
                                                                                                                                                 len(agent2.replay_buffer)),end="")
        torch.save(agent1.policy.state_dict(), run_dir / 'checkpoint_actor1.pth')
        torch.save(agent1.critic.state_dict(), run_dir / 'checkpoint_critic1.pth')
        torch.save(agent1.target_policy.state_dict(),run_dir / 'checkpoint_actor_target1.pth')
        torch.save(agent1.target_critic.state_dict(),run_dir / 'checkpoint_critic_target1.pth')
            
        torch.save(agent2.policy.state_dict(), run_dir /'checkpoint_actor2.pth')
        torch.save(agent2.critic.state_dict(),run_dir / 'checkpoint_critic2.pth')
        torch.save(agent2.target_policy.state_dict(),run_dir / 'checkpoint_actor_target2.pth')
        torch.save(agent2.target_critic.state_dict(),run_dir / 'checkpoint_critic_target2.pth')
    

    
        

Episode 0	 Mean Score of both Agents: 0.0000, replay memory0 11.000, repleay mem1 11.000 
Episode 200	 Mean Score of both Agents: 0.0020, replay memory0 3007.000, repleay mem1 3007.000 
Episode 400	 Mean Score of both Agents: 0.0010, replay memory0 6030.000, repleay mem1 6030.000 
Episode 600	 Mean Score of both Agents: 0.0000, replay memory0 8870.000, repleay mem1 8870.000 
Episode 800	 Mean Score of both Agents: 0.0000, replay memory0 11778.000, repleay mem1 11778.000 
Episode 1000	 Mean Score of both Agents: 0.0270, replay memory0 15526.000, repleay mem1 15526.000 
Episode 1200	 Mean Score of both Agents: 0.1930, replay memory0 22857.000, repleay mem1 22857.000 
Episode 1400	 Mean Score of both Agents: 0.3860, replay memory0 35583.000, repleay mem1 35583.000 
Episode 1600	 Mean Score of both Agents: 0.3710, replay memory0 50455.000, repleay mem1 50455.000 
Episode 1800	 Mean Score of both Agents: 1.0300, replay memory0 85968.000, repleay mem1 85968.000 
Episode 1900	 Mean Score of b

KeyboardInterrupt: 

In [ ]:

agent1 = DDPGAgent(**agent_init_params)
agent2 = DDPGAgent(**agent_init_params)

scores = []
mean_scores = []

agent1.policy.cuda()
agent1.target_policy.cuda()

t_step = 0

for i_episode in range(0, number_of_episodes):
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.vector_observations
    
    agent1.reset_noise()
    agent2.reset_noise()
    score = 0
    
    while True:
        state1 =  state[0]
        state2 = state[1] 
        
        action1 = agent1.act(state1.reshape((1,state1.shape[0])),explore=True)
        action2 = agent2.act(state2.reshape((1,state2.shape[0])),explore=True)
             
        env_info = env.step([action1, action2])[brain_name]
        next_state = env_info.vector_observations
        reward = env_info.rewards
        done = env_info.local_done
        
        next_state1 = next_state[0] 
        next_state2 = next_state[1] 
        
        agent1.step(state1, action1, reward[0],next_state1, done,t_step)
        agent2.step(state2, action2, reward[1],next_state2, done,t_step)
        
        state = next_state
        score += np.max(reward)
        t_step+=1
        
        if np.any(done):
            break
            
    scores.append(score)
    mean = np.mean(scores[-100:])

    print('\rEpisode {}\t Mean Score of both Agents: {:.4f}, replay memory0 {:.3f}, repleay mem1 {:.3f} '.format(i_episode,  mean,len(agent1.replay_buffer),
                                                                                                                                                 len(agent2.replay_buffer)),end="")
    #print('\rstep {}'.format(t_step),end="")
    if i_episode % 200 ==0:
        #logger.add_scalar('agent%i/mean_episode_rewards' % i_episode,  mean_score)
        print('\rEpisode {}\t Mean Score of both Agents: {:.4f}, replay memory0 {:.3f}, repleay mem1 {:.3f} '.format(i_episode,  mean,len(agent1.replay_buffer),
                                                                                                                                                 len(agent2.replay_buffer)))
    
    if i_episode % 100 ==0:
        print('\rEpisode {}\t Mean Score of both Agents: {:.4f}, replay memory0 {:.3f}, repleay mem1 {:.3f} '.format(i_episode,  mean,len(agent1.replay_buffer),
                                                                                                                                                 len(agent2.replay_buffer)),end="")
        torch.save(agent1.policy.state_dict(), run_dir / 'checkpoint_actor1.pth')
        torch.save(agent1.critic.state_dict(), run_dir / 'checkpoint_critic1.pth')
        torch.save(agent1.target_policy.state_dict(),run_dir / 'checkpoint_actor_target1.pth')
        torch.save(agent1.target_critic.state_dict(),run_dir / 'checkpoint_critic_target1.pth')
            
        torch.save(agent2.policy.state_dict(), run_dir /'checkpoint_actor2.pth')
        torch.save(agent2.critic.state_dict(),run_dir / 'checkpoint_critic2.pth')
        torch.save(agent2.target_policy.state_dict(),run_dir / 'checkpoint_actor_target2.pth')
        torch.save(agent2.target_critic.state_dict(),run_dir / 'checkpoint_critic_target2.pth')
    

    
        

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label='MADDPG')
plt.plot(np.arange(len(scores)), avgs, c='r', label='avg')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend(loc='upper left');
plt.show()

### 5. Test the model


In [8]:
agent1_pre = DDPGAgent(**agent_init_params)
agent2_pre = DDPGAgent(**agent_init_params)


# load the pretained model

agent_0_weights = run_dir  /  'checkpoint_actor1.pth'
agent_0_critic_weights =  run_dir  /  'checkpoint_critic1.pth'
agent_0_actor_target =  run_dir  /  'checkpoint_actor_target1.pth'
agent_0_critic_target =   run_dir  / 'checkpoint_critic_target1.pth'


agent_1_weights =  run_dir  /  'checkpoint_actor2.pth'
agent_1_critic_weights = run_dir  /  'checkpoint_critic2.pth'
agent_1_actor_target = run_dir  /   'checkpoint_actor_target2.pth'
agent_1_critic_target = run_dir  /   'checkpoint_critic_target2.pth'

agent1_pre.policy.load_state_dict(torch.load(agent_0_weights))
agent1_pre.critic.load_state_dict(torch.load(agent_0_critic_weights))
agent1_pre.target_policy.load_state_dict(torch.load(agent_0_actor_target))
agent1_pre.target_critic.load_state_dict(torch.load(agent_0_critic_target))

agent2_pre.policy.load_state_dict(torch.load(agent_1_weights))
agent2_pre.critic.load_state_dict(torch.load(agent_1_critic_weights))
agent2_pre.target_policy.load_state_dict(torch.load(agent_1_actor_target))
agent2_pre.target_critic.load_state_dict(torch.load(agent_1_critic_target))

In [11]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        
        state1 =  states[0]
        state2 = states[1] 
        
        action1 = agent1_pre.act(state1.reshape((1,state1.shape[0])),explore=False)
        action2 = agent2_pre.act(state2.reshape((1,state2.shape[0])),explore=False)
        
        env_info = env.step([action1, action2])[brain_name]
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

KeyboardInterrupt: 

In [12]:
env.close()